In [1]:
import numpy as np
import pandas as pd
import sys

In [2]:
from FE import *

In [3]:
sys.path.append('./tools/')

In [4]:
from mylgb_multiclass import Mylgb
from mylgb_reg import Mylgb_regression

# evaluation indicator 

In [5]:
def MAE(pred_y, true_y):
    return np.sum(np.abs(pred_y - true_y)) / pred_y.shape[0] / pred_y.shape[1]

In [6]:
def get_score(mae):
    return 1.0 / (1.0 + 10 * mae)

In [7]:
def get_mae(score):
    return  (1.0 / score - 1.0) / 10.0

In [8]:
def get_partition(x):
    result = np.zeros(4)
    for i in x:
        result[i] += 1
    return result / sum(result)

# EDA

In [16]:
for i in range(1, 11):
    tr_data['Attribute' + str(i)] = np.array(log_trans(tr_data['Attribute' + str(i)].values, 10) * 10, dtype=int)

# preprocess

In [9]:
dict_label2id = {
    'Excellent':0, 'Good':1, 'Pass':2, 'Fail':3
}

In [10]:
# read data

In [11]:
tr_data = pd.read_csv('./rawdata/first_round_training_data.csv', encoding='utf-8')

In [12]:
te_data = pd.read_csv('./rawdata/first_round_testing_data.csv', encoding='utf-8')

In [13]:
tr_data['Quality_label'] = tr_data['Quality_label'].apply(lambda x: dict_label2id[x])

In [14]:
tr_fea_names = list(tr_data.columns)
te_fea_names = list(tr_fea_names[:10])

In [15]:
tr_data['Group'] = te_data['Group'].values

# add new column

[]

In [25]:
tr_new_col = []
te_new_col = []
for i in range(4,11):
    config = {
        'df_train' : tr_data,
        'base_feature' : te_fea_names,
        'id_name' : 'id',
        'label_name' : 'Attribute' + str(i)
    }
    tmp_model = Mylgb_regression(config)
    tmp_model.train()
    tr_new_col.append(tmp_model.model.predict(tr_data[te_fea_names]))
    te_new_col.append(tmp_model.model.predict(te_data[te_fea_names]))

/home/betago/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[100]	cv_agg's l1: 7.42055 + 0.313722
[200]	cv_agg's l1: 6.95594 + 0.281447
[300]	cv_agg's l1: 6.83576 + 0.26237
[400]	cv_agg's l1: 6.79542 + 0.254375
[500]	cv_agg's l1: 6.77977 + 0.251074
[600]	cv_agg's l1: 6.77354 + 0.249073
[700]	cv_agg's l1: 6.76901 + 0.245924
[800]	cv_agg's l1: 6.76549 + 0.245717
[900]	cv_agg's l1: 6.76258 + 0.245153
[100]	cv_agg's l1: 6.76492 + 0.287286
[200]	cv_agg's l1: 6.17564 + 0.221813
[300]	cv_agg's l1: 6.03581 + 0.190461
[400]	cv_agg's l1: 5.99512 + 0.180067
[500]	cv_agg's l1: 5.9841 + 0.177222
[600]	cv_agg's l1: 5.98116 + 0.173706
[700]	cv_agg's l1: 5.98105 + 0.169186
[100]	cv_agg's l1: 7.26496 + 0.307525
[200]	cv_agg's l1: 6.70191 + 0.29158
[300]	cv_agg's l1: 6.5987 + 0.282531
[400]	cv_agg's l1: 6.57858 + 0.28175
[500]	cv_agg's l1: 6.57256 + 0.280533
[600]	cv_agg's l1: 6.57211 + 0.279402
[100]	cv_agg's l1: 8.99489 + 0.348143
[200]	cv_agg's l1: 8.90928 + 0.3206
[300]	cv_agg's l1: 8.88435 + 0.31112
[400]	cv_agg's l1: 8.88374 + 0.309253
[100]	cv_agg's l1: 8

In [28]:
len(tr_new_col)

7

In [29]:
for i in range(7):
    tr_data['new_f' + str(i)] = tr_new_col[i]
    te_data['new_f' + str(i)] = te_new_col[i]

In [31]:
new_fename = ['new_f' + str(i) for i in range(7)]

# predict

In [36]:
config = {
    'df_train' : tr_data,
    'base_feature' : new_fename,
    'id_name' : 'id',
    'label_name' : 'Quality_label'
}

In [37]:
lgb_model_1 = Mylgb(config)

In [38]:
lgb_model_1.train()

[100]	cv_agg's multi_logloss: 1.09862 + 0.00958445
[200]	cv_agg's multi_logloss: 1.02288 + 0.0143345
[300]	cv_agg's multi_logloss: 1.00214 + 0.0178668
[400]	cv_agg's multi_logloss: 0.998825 + 0.0201715


0.9987652086620574

In [39]:
tr_pred_prob = lgb_model_1.model.predict(tr_data[new_fename])

In [40]:
tr_pred = np.argmax(tr_pred_prob, 1)

In [41]:
tr_group = tr_data['Group'].values

In [42]:
tr_label = tr_data['Quality_label'].values

In [43]:
tr_label

array([2, 3, 3, ..., 2, 2, 2])

In [44]:
need_pred_true = []
my_pred_prob = []
my_pred_notprob = []
for i in range(120):
    need_pred_true.append(get_partition(tr_label[tr_group == i]))
    my_pred_prob.append(np.sum(tr_pred_prob[tr_group == i], axis=0))
    my_pred_notprob.append(get_partition(tr_pred[tr_group == i]))

In [45]:
need_pred_true = np.array(need_pred_true)

In [46]:
my_pred_notprob = np.array(my_pred_notprob)

In [47]:
my_pred_prob = np.array([i / sum(i) for i in my_pred_prob])

In [48]:
get_score(MAE(need_pred_true, my_pred_notprob))

0.5899705014749261

In [49]:
get_score(MAE(need_pred_true, my_pred_prob))

0.7441137367103127

In [94]:
tr_pred_part = get_partition(tr_pred[50:100])

In [95]:
tr_true_part = get_partition(tr_data['Quality_label'].values[50:100])

In [96]:
tr_true_part

array([0.24, 0.24, 0.4 , 0.12])

In [97]:
tr_pred_part

array([0.08, 0.08, 0.78, 0.06])

In [98]:
get_score(MAE(tr_pred_part, tr_true_part))

0.3448275862068966

In [101]:
get_score(MAE(tr_pred_part_new, tr_true_part))

0.618786182008356

In [99]:
tr_pred_part_new = np.sum(tr_pred_prob[50:100], 0) / sum(np.sum(tr_pred_prob[50:100], 0))

In [100]:
tr_pred_part_new

array([0.16580873, 0.19230192, 0.52321342, 0.11867593])

# submit

In [50]:
submit_pd = pd.read_csv('./rawdata/submit_example.csv', encoding='utf-8', index_col=None)

In [51]:
te_pred_prob = lgb_model_1.model.predict(te_data[new_fename])

In [53]:
te_pred_prob

array([[0.49027116, 0.18413939, 0.28126662, 0.04432283],
       [0.0570145 , 0.35247445, 0.13977326, 0.45073778],
       [0.05099459, 0.44129674, 0.1177373 , 0.38997136],
       ...,
       [0.4652235 , 0.30822709, 0.1902598 , 0.03628961],
       [0.10817755, 0.58356138, 0.1767927 , 0.13146837],
       [0.50975179, 0.24745654, 0.19610309, 0.04668858]])

In [73]:
te_pred = np.argmax(te_pred_prob, 1)

In [74]:
te_pred

array([0, 3, 3, ..., 2, 1, 1])

In [102]:
te_pred_prob

array([[0.51108102, 0.24781982, 0.17547253, 0.06562663],
       [0.05445464, 0.25160678, 0.17330528, 0.52063331],
       [0.07212852, 0.22876436, 0.19888323, 0.50022388],
       ...,
       [0.2562444 , 0.29993487, 0.4035007 , 0.04032003],
       [0.11724435, 0.50114796, 0.09168262, 0.28992507],
       [0.28683289, 0.42900776, 0.20224147, 0.08191787]])

In [54]:
group_value = te_data['Group'].values

In [55]:
final_result = []
for i in range(120):
    final_result.append(np.sum(te_pred_prob[group_value == i], axis = 0))

In [56]:
final_result = [i / sum(i) for i in final_result]

In [57]:
final_result = np.array(final_result)

In [58]:
submit_pd[['Excellent ratio', 'Good ratio', 'Pass ratio', 'Fail ratio']] = final_result

In [59]:
submit_pd.to_csv('lgb_pred_attribute.csv', index=None)

In [114]:
print(pd.DataFrame({
        'column': lgb_model_1.model.feature_name(),
        'importance': lgb_model_1.model.feature_importance(),
    }).sort_values(by='importance'))

     column  importance
6  Column_6        2180
7  Column_7        2916
8  Column_8        3160
5  Column_5        5910
4  Column_4        7700
1  Column_1        8851
2  Column_2        9084
0  Column_0        9148
3  Column_3        9620
9  Column_9       10227


In [115]:
print(pd.DataFrame({
        'column': lgb_model_2.model.feature_name(),
        'importance': lgb_model_2.model.feature_importance(),
    }).sort_values(by='importance'))

     column  importance
0  Column_0        4408
1  Column_1        4688
2  Column_2        4937
4  Column_4       14559
3  Column_3       15359
7  Column_7       17195
9  Column_9       18278
5  Column_5       20620
8  Column_8       21853
6  Column_6       23821
